In [27]:
import os
import pretty_midi

for filename in os.listdir("test_dataset"):
    try:
        midi_data = pretty_midi.PrettyMIDI("test_dataset/" + filename)
    except:
        continue
    
    f = open('index_dataset/' + filename + '.txt', 'w')
    print(filename)
    
    #first find the song duration
    song_duration = midi_data.get_end_time()
    
    
    #find tempo 
    tempo = midi_data.estimate_tempo()
    print("Tempo: " + str(tempo))
    #f.write(str(tempo) + "\n")
    
    segment_duration = 15
    
    #in each segment in sliding window write the notes to a file
    for i in range(0, int(song_duration)-segment_duration):
        for note in midi_data.instruments[0].notes:
            if note.start > i and note.end < i + segment_duration:
                f.write(str(note.pitch) + " ")
        f.write("\n")
        
    f.close()
    


Angel_Eyes.mid
Tempo: 177.98377752028006
Dancing_Queen.mid
Tempo: 210.25434498609175
Danza_Ungherese_n._8_in_la_minore-Presto.mid
Tempo: 224.7252824363204
Everybody_Backstreets_Back_.1.mid
Tempo: 205.3341258303746
For_Your_Love.mid
Tempo: 194.7963113011656
Ill_Be_There_For_You.mid
Tempo: 213.94133967876826
Its_OK.mid
Tempo: 234.08229711785214
Joan_of_Arc_Maid_of_Orleans_.mid
Tempo: 220.87110005189152
Never_Gonna_Give_You_Up.1.mid
Tempo: 235.89881499508306
ThankYou.mid
Tempo: 160.0
The_Real_Slim_Shady.mid
Tempo: 210.00315004725013
Words.mid
Tempo: 172.5235562880708


In [33]:
import os
import pretty_midi

for filename in os.listdir("test_dataset"):
    try:
        midi_data = pretty_midi.PrettyMIDI("test_dataset/" + filename)
    except:
        continue
    
    
    print("File: " + filename)
    
    #first find the song duration
    song_duration = midi_data.get_end_time()
    print("Song duration: " + str(song_duration))
    
    #find the number of notes in the song
    print("Number of notes: " + str(len(midi_data.instruments[0].notes)))
    
    print(midi_data.instruments[0].notes)
    
    segment_duration = 30
    
    f = open('index_dataset/' + filename + '.txt', 'w')
    
    #in each segment in sliding window write the notes to a file
    
    # for i in range(0, int(song_duration)-segment_duration):
    #     prev = 1
    #     for j in range(prev, len(midi_data.instruments[0].notes)):
    #         if midi_data.instruments[0].notes[i].start > i and midi_data.instruments[0].notes[i].end < i+segment_duration:
    #             #relative pitches should be written
    #             f.write(str(abs(midi_data.instruments[0].notes[i].pitch - midi_data.instruments[0].notes[i-1].pitch)) + " ")
    #         f.write("\n")
    #     else:
    #         prev = j
    #         continue
        
    # f.close()
    
    for i in range(0, int(song_duration) - segment_duration):
        prev = 1
        for j in range(prev, len(midi_data.instruments[0].notes)):
            note = midi_data.instruments[0].notes[j]
            if note.start > i and note.end < i + segment_duration:
                # relative pitches should be written
                f.write(str(abs(note.pitch - midi_data.instruments[0].notes[j - 1].pitch)) + " ")
        f.write("\n")
        prev = j + 1

    f.close()

File: Angel_Eyes.mid
Song duration: 219.0
Number of notes: 944
[Note(start=6.000000, end=6.750000, pitch=42, velocity=90), Note(start=6.750000, end=7.250000, pitch=37, velocity=74), Note(start=6.750000, end=7.250000, pitch=42, velocity=100), Note(start=7.250000, end=7.500000, pitch=42, velocity=80), Note(start=7.500000, end=8.000000, pitch=42, velocity=90), Note(start=8.250000, end=8.750000, pitch=37, velocity=74), Note(start=8.250000, end=8.750000, pitch=42, velocity=100), Note(start=8.750000, end=9.000000, pitch=42, velocity=80), Note(start=9.000000, end=9.750000, pitch=42, velocity=90), Note(start=9.750000, end=10.250000, pitch=37, velocity=74), Note(start=9.750000, end=10.250000, pitch=42, velocity=100), Note(start=10.250000, end=10.500000, pitch=42, velocity=80), Note(start=10.500000, end=11.000000, pitch=42, velocity=90), Note(start=11.000000, end=11.250000, pitch=46, velocity=90), Note(start=11.250000, end=11.750000, pitch=37, velocity=74), Note(start=11.250000, end=11.750000, p

In [54]:
midi = pretty_midi.PrettyMIDI('test_dataset/Words.mid')

for i in range(0,11):
    notes = midi.instruments[i].notes
    
    #save the notes in new midi file
    new_midi = pretty_midi.PrettyMIDI()
    new_midi.instruments.append(pretty_midi.Instrument(0))
    for note in notes:
        new_midi.instruments[0].notes.append(note)
    new_midi.write('test_dataset/Words' + str(i) + '.mid')




In [28]:
from fastdtw import fastdtw
from dtw import utw

def get_dtw_distance(input_pitches, segment_pitches):
    distance,path= fastdtw(input_pitches, segment_pitches)
    
    return distance

In [30]:
input_midi_data = pretty_midi.PrettyMIDI("input.mid")
input_pitches=[]
# for i in range(1, len(input_midi_data.instruments[0].notes)):
#     input_pitches.append(abs(input_midi_data.instruments[0].notes[i].pitch - input_midi_data.instruments[0].notes[i-1].pitch))

for note in input_midi_data.instruments[0].notes:
    input_pitches.append(note.pitch)
    
    
print("Input pitches: " + str(input_pitches))

top_5 = []

for filename in os.listdir("index_dataset"):
    smallest = 10000000
    best_segment = None
    f = open('index_dataset/' + filename, 'r')
    content = f.read()
    notes = content.split("\n")
    
    for note in notes:
        if note == '':
            continue
        #convert string to array of integers
        
        note = note.split(" ")
        #remove last element
        note.pop()
        note = list(map(int, note))
        distance = get_dtw_distance(input_pitches, note)
        if distance < smallest:
            smallest = distance
            best_segment = note
    
    print("File: " + filename)
    print("Distance: " + str(smallest))
    print("Segment: " + str(best_segment))        
    #top_5.append({"filename": filename, "distance": smallest, "segment": best_segment})
    
    
    f.close()
#top_5.sort(key=lambda x: x["distance"])
# for i in range(0, 5):
#     print("File: " + top_5[i]["filename"])
#     print("Distance: " + str(top_5[i]["distance"]))
#     print("Segment: " + str(top_5[i]["segment"]))
#     print("\n")
    
        

Input pitches: [53, 53, 53, 54, 52, 60, 48, 52, 48, 50, 48, 52, 48, 53, 53, 53, 51, 49, 53, 56, 57, 52, 53, 53, 53, 57, 56, 56, 56, 57, 54, 51]
File: Angel_Eyes.mid.txt
Distance: 116.0
Segment: [50, 48, 46, 46, 50, 45, 45, 55, 50, 53, 44, 45, 39, 38]
File: Dancing_Queen.mid.txt
Distance: 226.0
Segment: [64, 61, 64, 64, 64, 64, 64, 66, 61, 64, 64, 64, 64, 64, 61, 61, 59, 57, 59, 59]
File: Danza_Ungherese_n._8_in_la_minore-Presto.mid.txt
Distance: 340.0
Segment: [38, 43, 36, 43, 38, 43, 36, 43, 38, 42, 38, 43, 38, 43, 38, 43, 38, 43, 36, 43, 38, 38, 50, 43, 43, 31]
File: Everybody_Backstreets_Back_.1.mid.txt
Distance: 123.0
Segment: [36, 49, 54, 42, 36, 49, 54, 54, 56, 42, 80, 54, 56]
File: For_Your_Love.mid.txt
Distance: 267.0
Segment: [44, 42, 51, 44, 40, 51, 53, 42, 51, 44, 51, 42, 35, 53, 44, 35, 51, 40, 51, 42, 35, 51, 53, 44, 42, 51, 46, 40, 51, 35, 40, 51, 35, 51, 53, 42, 35, 51]
File: Ill_Be_There_For_You.mid.txt
Distance: 70.0
Segment: [52, 57, 59, 62]
File: Its_OK.mid.txt
Dista

In [20]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200/",
    ca_certs="C:\Elastic\elasticsearch-8.11.1\config\certs\http_ca.crt",
    basic_auth=("elastic", "PgrlijTLFLyC9X3Y*pRa")
)

print(es.ping())


True


In [21]:
#create index
mapping = {
    "mappings": {
        "properties": {
            'song': { 'type': 'text' },
            'notes': { 'type': 'text'},
            'tempo': { 'type': 'float'},
            'relative_pitches': { 'type': 'text'}
        }
    }
}

if(es.indices.exists(index="midi")):
    es.indices.delete(index="midi")

es.indices.create(index="midi", body=mapping, ignore=400)

C:\Users\soumil\AppData\Local\Temp\ipykernel_4432\2734717090.py:16: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="midi", body=mapping, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'midi'})

In [ ]:
for filename in os.listdir("index_dataset"):
    f = open("index_dataset/" + filename, "r")
    content = f.read()
    
    #each line should be a document
    line = content.split("\n")
    for i in range(0, len(line)):
        if (line[i]== ""):
            continue
        doc = {
            "song": filename,
            "notes": line[i]
        }
        es.index(index="midi", body=doc)
    
    print(filename + " indexed")
    
    f.close()

In [ ]:
from fastdtw import fastdtw

def get_dtw_distance(input_pitches, segment_pitches):
    distance, path = fastdtw(input_pitches, segment_pitches)
    
    return distance

In [ ]:
input_midi_data = pretty_midi.PrettyMIDI("input.mid")
print(input_midi_data.estimate_tempo())

In [ ]:
#given a query mid file, return the most similar mid file

#use cosine similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

input_midi_data = pretty_midi.PrettyMIDI("input.mid")
input_pitches=[]
for note in input_midi_data.instruments[0].notes:
    input_pitches.append(note.pitch)

query = {
        'query': {
            'function_score': {
                'functions': [
                    {
                        'script_score': {
                            'script': {
                                'source': 'fastdtw(params.query_pitches, doc["notes"])',
                                'params': {
                                    'query_pitches': input_pitches
                                }
                            }
                        }
                    }
                ],
                'score_mode': 'avg'
            }
        }
}

result = es.search(index='midi', body=query)
print(result['hits']['hits'][0:3]['_source']['song']) 



In [ ]:
#given a query mid file, return the most similar mid file

#use cosine similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

input_midi_data = pretty_midi.PrettyMIDI("input.mid")
input_pitches=[]
for note in input_midi_data.instruments[0].notes:
    input_pitches.append(note.pitch)


dtw_script = {
        "script": {
            "source": "1.0 / (1.0 + get_dtw_distance(params.query_sequence, doc['features'].notes))",
            "lang": "painless",
        },
        "params": {"query_sequence": input_pitches},
    }

query = {
        "query": {
            "function_score": {
                "query": {"match_all": {}},
                "script_score": dtw_script
            }
        }
    }


results = es.search(index='midi', body=query)
print(results['hits']['hits'][:3])
